# Evaluate FlowNet and DispNet for disparity evaluation

To evaluate the FlowNetS, FlowNetC, DispNetS and DispNetC for disparity evaluation on FlyingThings3D, we use the function eval() from the file eval.py. 

In [ ]:
from eval import evaluate, setup_args
import pandas as pd
import numpy as np
from pathlib import Path
import sys

sys.argv = [""]


## Evaluation command

The following block will run the evaluation with the provided pre-trained weights on the FlyingThings3D test set. Note that the evaluation may run for several minutes. Have a look at the qualitative results by using tensorboard on the specified output directory.

The function will try to load existing results instead of running the evaluation. To run the evaluations in the **command line instead of the notebook**, run:

~~~bash
python eval.py --model FlowNetS --auto_restore mw/flownets --output output/flownets
python eval.py --model FlowNetC --auto_restore mw/flownetc --output output/flownetc
python eval.py --model DispNetS --auto_restore mw/dispnets --output output/dispnets
python eval.py --model DispNetC --auto_restore mw/dispnetc --output output/dispnetc
~~~

Then download the results and move them s.t. they are found by the `run_or_get_output` function below.

In [ ]:
OUTPUT_DIR = "output"


def run_or_get_output(args):
    output_dir = Path(args.output) / "eval" / "FlyingThings3D"
    metrics_file = output_dir / "metrics.csv"
    results_file = output_dir / "results.csv"
    if metrics_file.is_file() and results_file.is_file():
        print(f"Loading results from {output_dir}")
        metrics = pd.read_csv(metrics_file, index_col=0)
        # pandas save Series as DataFrame but we want to load it as Series again
        results_df = pd.read_csv(results_file, index_col=0)
        indx = results_df.index
        vals = [results_df.iloc[i, 0] for i in range(len(indx))]
        results = pd.Series(vals, index=indx)
    else:
        print(f"Computing results with arguments {args}")
        metrics, results = evaluate(args)
    return metrics, results


## FlowNetS Evaluation

In [ ]:
args = setup_args()
args.model = "FlowNetS"
args.auto_restore = "mw/flownets"
args.output = f"{OUTPUT_DIR}/flownets"
flownet_s_metrics, flownet_s_results = run_or_get_output(args)


The metrics for each sample and the averaged results are returned as pandas Seris and Dataframes, which can be directly displayed by Jupyter notebook as tables and be easily plotted with matplotlib. 

In [ ]:
flownet_s_metrics


In [ ]:
flownet_s_results


In [ ]:
flownet_s_metrics.plot()


## FlowNetC Evaluation

In [ ]:
args = setup_args()
args.model = "FlowNetC"
args.auto_restore = "mw/flownetc"
args.output = f"{OUTPUT_DIR}/flownetc"
flownet_c_metrics, flownet_c_results = run_or_get_output(args)


In [ ]:
print(flownet_c_results)


## DispNetS Evaluation

In [ ]:
args = setup_args()
args.model = "DispNetS"
args.auto_restore = "mw/dispnets"
args.output = f"{OUTPUT_DIR}/dispnets"
dispnet_s_metrics, dispnet_s_results = run_or_get_output(args)


In [ ]:
print(dispnet_s_results)


## DispNetC Evaluation

In [ ]:
args = setup_args()
args.model = "DispNetC"
args.auto_restore = "mw/dispnetc"
args.output = f"{OUTPUT_DIR}/dispnetc"
dispnet_c_metrics, dispnet_c_results = run_or_get_output(args)


If your correlation layer implementation from the previous task was correct, this should give an EPE of 1.67.

In [ ]:
print(dispnet_c_results)


## Comparison

Now that results are available for all four models, we want to compare results. Your task is to show the results as table and plot them as shown on the assignment sheet.
For this, you have to combine the pandas Series/DataFrames that contain the metrics/results of each model. Have a look at the pandas documentation and make use of the notebook to quickly try and check commands. 

In [ ]:
flownet_s_results


In [ ]:
{
    "FlowNetS": flownet_s_results,
    "FlowNetC": flownet_c_results,
    "DispNetS": dispnet_s_results,
    "DispNetC": dispnet_c_results,
}


In [ ]:
# START TODO #################
# Combine the flownet_s_metrics, flownet_c_metrics, dispnet_s_metrics and dispnet_c_metrics
# to a pandas DataFrame called combined_metrics.
# Combine the flownet_s_results, flownet_c_results, dispnet_s_results and dispnet_c_results
# to a pandas DataFrame called combined_results.
# combined_metrics = ...
# combined_results = ...
raise NotImplementedError
# END TODO #################


Show the combined metrics:

In [ ]:
combined_metrics


In [ ]:
combined_metrics.plot(ylabel="EPE", ylim=(0, 30))


Show the combined results:

In [ ]:
combined_results


In [ ]:
# START TODO #################
# Create the bar plot as shown in Figure 3b of the assignment sheet.
raise NotImplementedError
# END TODO #################


## Rectification

See the assignment PDF for details, chapter *Apply DispNet to images from a monocular camera*.

In [ ]:
import os.path as osp
from importlib import reload
from PIL import Image
from lib.vis import np2d, np3d
import lib.utils as utils


In [ ]:
def load_data(path):
    path_l = osp.join(path, "left.png")
    path_r = osp.join(path, "right.png")

    image_l = np.array(Image.open(path_l)).transpose([2, 0, 1])  # 3, H, W
    image_r = np.array(Image.open(path_r)).transpose([2, 0, 1])  # 3, H, W
    _, h_orig, w_orig = image_l.shape

    K = np.load(osp.join(path, "K.npy"))
    r_to_l_transform = np.load(osp.join(path, "right_to_left_transform.npy"))
    return image_l, image_r, K, r_to_l_transform


First we implement some basic functions for:
- projecting a 3d point to an image
- computing the epipole from another camera in a current image
- computing the essential and fundamental matrices

We will test these functions on multiple samples from different datasets in the following.

### KITTI 1

In [ ]:
path = "data/kitti"
image_l, image_r, K, r_to_l_transform = load_data(path)


First we simply visualize the images:

In [ ]:
np3d(np.concatenate([image_l, image_r], -1), image_range_text_off=True)


Note that the images are actually not left and right images from a stereo camera pair! Instead the images are from a video with forward motion between the first and second image. We still call the first image image_l and the second image image_r, as this is how the DispNet will interpret the images.

Next we compute the epipoles in both images and visualize them in the images.

In [ ]:
reload(utils)
epi_in_r = utils.get_epipole(K, r_to_l_transform)
epi_in_l = utils.get_epipole(K, utils.invert_transform(r_to_l_transform))


In [ ]:
np3d(
    image_l,
    markers=[{"xy_pos": epi_in_l, "desc": "epipole"}],
    text="Image 1 with epipole from image 2",
    image_range_text_off=True,
)


In [ ]:
np3d(
    image_r,
    markers=[{"xy_pos": epi_in_r, "desc": "epipole"}],
    text="Image 2 with epipole from image 1",
    image_range_text_off=True,
)


Note that the epipole from image 1 is within the image plane of image 2 even though the camera from image 2 is *behind* the image plane of image 1. This simply is because mathematically also points behind the image plane can be projected onto the image plane and we did not check this.

Next we can visualize some epipolar lines in the second image. For this, we need to compute the fundamental matrix from the known camera calibration. We call the fundamental matrix F_l_to_r, as it gives us an epipolar line l=Fx in the right (=second) image given a point x in the left (=first) image.

In [ ]:
reload(utils)
F_l_to_r = utils.compute_fundamental_matrix(K, r_to_l_transform)
F_l_to_r


It is also possible to compute the fundamental matrix from the essential matrix. Here, we use this as a sanity check:

In [ ]:
reload(utils)
E_l_to_r = utils.compute_essential_matrix(r_to_l_transform)
F_l_to_r_check = np.linalg.inv(K.T).dot(E_l_to_r).dot(np.linalg.inv(K))
F_l_to_r_check


On first glance the fundamental matrix that was computed from the essential matrix looks different. But keep in mind that the fundamental matrix is defined only up to scale, so it is okay if both matrices differ by a fixed scale factor:

In [ ]:
F_l_to_r_check / F_l_to_r


We can now use the fundamental matrix to visualize epipolar lines in the second image for reference points in the first image:

In [ ]:
reload(utils)
x_ref_0 = (96, 130)
x_ref_1 = (290, 300)
color_0 = (0, 255, 0)
color_1 = (0, 0, 255)
image_r_epilines = utils.plot_epipolar_line(
    image_r, F_l_to_r, x_ref=x_ref_0, line_color=color_0
)
image_r_epilines = utils.plot_epipolar_line(
    image_r_epilines, F_l_to_r, x_ref=x_ref_1, line_color=color_1
)


In [ ]:
np3d(
    image_l,
    markers=[
        {"xy_pos": epi_in_l, "desc": "epipole"},
        {"xy_pos": x_ref_0, "desc": "x_ref_0", "marker_color": color_0},
        {"xy_pos": x_ref_1, "desc": "x_ref_1", "marker_color": color_1},
    ],
    text="Image 1 with epipole from image 2",
    image_range_text_off=True,
)


In [ ]:
np3d(
    image_r_epilines,
    markers=[{"xy_pos": epi_in_r, "desc": "epipole"}],
    text="Image 2 with epipole from image 1 and epipolar lines for two reference points",
    image_range_text_off=True,
)


You can visualize check that the epipolar lines cross the corresponding points of the two referents points in the first image.

### KITTI 2

In [ ]:
path = "data/kitti_2"
image_l, image_r, K, r_to_l_transform = load_data(path)
np3d(np.concatenate([image_l, image_r], -1), image_range_text_off=True)


In [ ]:
epi_in_r = utils.get_epipole(K, r_to_l_transform)
epi_in_l = utils.get_epipole(K, utils.invert_transform(r_to_l_transform))
F_l_to_r = utils.compute_fundamental_matrix(K, r_to_l_transform)


In [ ]:
reload(utils)
x_ref_0 = (310, 230)
x_ref_1 = (480, 240)
color_0 = (0, 255, 0)
color_1 = (0, 0, 255)
image_r_epilines = utils.plot_epipolar_line(
    image_r, F_l_to_r, x_ref=x_ref_0, line_color=color_0
)
image_r_epilines = utils.plot_epipolar_line(
    image_r_epilines, F_l_to_r, x_ref=x_ref_1, line_color=color_1
)


In [ ]:
np3d(
    image_l,
    markers=[
        {"xy_pos": epi_in_l, "desc": "epipole"},
        {"xy_pos": x_ref_0, "desc": "x_ref_0", "marker_color": color_0},
        {"xy_pos": x_ref_1, "desc": "x_ref_1", "marker_color": color_1},
    ],
    text="Image 1 with epipole from image 2",
    image_range_text_off=True,
)


In [ ]:
np3d(
    image_r_epilines,
    markers=[{"xy_pos": epi_in_r, "desc": "epipole"}],
    text="Image 2 with epipole from image 1 and epipolar lines for two reference points",
    image_range_text_off=True,
)


### RealThings

In [ ]:
path = "data/realthings"
image_l, image_r, K, r_to_l_transform = load_data(path)
np3d(np.concatenate([image_l, image_r], -1), image_range_text_off=True)


In [ ]:
epi_in_r = utils.get_epipole(K, r_to_l_transform)
epi_in_l = utils.get_epipole(K, utils.invert_transform(r_to_l_transform))
F_l_to_r = utils.compute_fundamental_matrix(K, r_to_l_transform)


In [ ]:
reload(utils)
x_ref_0 = (480, 200)
x_ref_1 = (690, 370)
color_0 = (0, 255, 0)
color_1 = (0, 0, 255)
image_r_epilines = utils.plot_epipolar_line(
    image_r, F_l_to_r, x_ref=x_ref_0, line_color=color_0
)
image_r_epilines = utils.plot_epipolar_line(
    image_r_epilines, F_l_to_r, x_ref=x_ref_1, line_color=color_1
)


In [ ]:
np3d(
    image_l,
    markers=[
        {"xy_pos": epi_in_l, "desc": "epipole"},
        {"xy_pos": x_ref_0, "desc": "x_ref_0", "marker_color": color_0},
        {"xy_pos": x_ref_1, "desc": "x_ref_1", "marker_color": color_1},
    ],
    text="Image 1",
    image_range_text_off=True,
)


In [ ]:
np3d(
    image_r_epilines,
    markers=[{"xy_pos": epi_in_r, "desc": "epipole"}],
    text="Image 2 with epipolar lines for two reference points",
    image_range_text_off=True,
)


Note that for this image pair the epipoles are out of the image boundaries:

In [ ]:
epi_in_l


In [ ]:
epi_in_r


### Rectification

We will work with the RealThings image pair for the following rectification.

In [ ]:
reload(utils)
image_l_rect, image_r_rect, H_l, H_r, rrect_to_lrect_transform = utils.rectify_images(
    image_l, image_r, K, r_to_l_transform
)


In [ ]:
np3d(np.concatenate([image_l_rect, image_r_rect], -1), image_range_text_off=True)


Next we can verify that epipolar lines are horizontal by visualizing the epipolar lines for the same reference points as before. Note that the reference point coordinates also changed, as the left image was warped too:

We can check that after rectification the epipoles are at infinity (i.e. (f, 0, 0) in homogeneous coordinates) and the fundamental matrix is of the form i_x where i=(1, 0, 0) and the i_x is the corresponding skew-symmetric matrix that represents a cross product:

In [ ]:
epi_in_r = utils.get_epipole(K, rrect_to_lrect_transform, return_hom=True)
epi_in_l = utils.get_epipole(
    K, utils.invert_transform(rrect_to_lrect_transform), return_hom=True
)
F_lrect_to_rrect = utils.compute_fundamental_matrix(K, rrect_to_lrect_transform)

print("Epipole in left image: {}.".format(epi_in_l))
print("Epipole in right image: {}.".format(epi_in_r))
print("Fundamental matrix: \n{}.".format(F_lrect_to_rrect / F_lrect_to_rrect[1, 2]))


In [ ]:
x_ref_0_rect = H_l.dot(np.append(np.array(x_ref_0), 1))
x_ref_0_rect = np.array(
    [x_ref_0_rect[0] / x_ref_0_rect[2], x_ref_0_rect[1] / x_ref_0_rect[2]]
)
x_ref_1_rect = H_l.dot(np.append(np.array(x_ref_1), 1))
x_ref_1_rect = np.array(
    [x_ref_1_rect[0] / x_ref_1_rect[2], x_ref_1_rect[1] / x_ref_1_rect[2]]
)
print(
    "Reference point 0 was at {} and is at {} after rectification.".format(
        x_ref_0, x_ref_0_rect
    )
)
print(
    "Reference point 1 was at {} and is at {} after rectification.".format(
        x_ref_1, x_ref_1_rect
    )
)

image_r_rect_epilines = utils.plot_epipolar_line(
    image_r_rect, F_lrect_to_rrect, x_ref=x_ref_0_rect, line_color=color_0
)
image_r_rect_epilines = utils.plot_epipolar_line(
    image_r_rect_epilines, F_lrect_to_rrect, x_ref=x_ref_1_rect, line_color=color_1
)


In [ ]:
np3d(
    image_l_rect,
    markers=[
        {"xy_pos": x_ref_0_rect, "desc": "x_ref_0", "marker_color": color_0},
        {"xy_pos": x_ref_1_rect, "desc": "x_ref_1", "marker_color": color_1},
    ],
    text="Image 1",
    image_range_text_off=True,
)


In [ ]:
np3d(
    image_r_rect_epilines,
    text="Image 2 with epipolar lines for two reference points",
    image_range_text_off=True,
)
